In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import pandas as pd
from collections import Counter
import re
import pickle
import MeCab
mc = MeCab.Tagger("-Ochasen")

In [2]:
# data_excel = pd.ExcelFile("data/excel2/new_pred_2.xlsx")
data_excel = pd.ExcelFile("data/excel2/new_pred_1.xlsx")
sheet_name = data_excel.sheet_names[0]
data_df = data_excel.parse(sheet_name)
data_df

,id,reply,group_id,cname,body,Espistemic,Argumentation,Coordination,Social
0,10730,0,1,飼育係,よろしくー\n\n,off task,non-argumentative moves,blank,externalization
1,10749,0,1,ドラ,よろしくお願いします～,off task,non-argumentative moves,blank,externalization
2,10754,0,1,海の日,よろしくお願いしまーす,off task,non-argumentative moves,blank,externalization
3,10849,0,1,飼育係,なんかやり方違うｗｗｗｗｗｗｗｗｗｗｗｗｗｗ,on task,non-argumentative moves,blank,externalization
4,10903,0,1,飼育係,拝見しましたが特に問題はないです,on task,non-argumentative moves,blank,quick consensus building
5,10908,0,1,ドラ,とりあえずこのまま進めてみますか？,on task,non-argumentative moves,blank,elicitation
6,10923,0,1,飼育係,そうしてくれると助かります,off task,non-argumentative moves,blank,externalization
7,11065,0,1,ドラ,お二人のワード拝見しました。\n\n内容については特に疑問点はありませんでした。,on task,non-argumentative moves,blank,quick consensus building
8,11070,0,1,ドラ,お二人のワード拝見しました。\n\n内容については特に疑問点はありませんでした。,on task,non-argumentative moves,blank,quick consensus building
9,11135,0,1,海の日,私も特にありませんでした。,on task,non-argumentative moves,technical coordination,externalization


In [3]:
data_df_L = data_df.drop(['id','reply','body'],axis=1)
data = data_df_L.values
data

array([[1, '飼育係', 'off task', 'non-argumentative moves', 'blank',
        'externalization'],
       [1, 'ドラ', 'off task', 'non-argumentative moves', 'blank',
        'externalization'],
       [1, '海の日', 'off task', 'non-argumentative moves', 'blank',
        'externalization'],
       ...,
       [46, 'イイダイ', 'off task', 'simple claim', 'technical coordination',
        'externalization'],
       [46, 'アキ', 'on task', 'non-argumentative moves', 'blank',
        'externalization'],
       [46, 'ジャスミン', 'on task', 'non-argumentative moves', 'blank',
        'externalization']], dtype=object)

In [4]:
sheet_name = data_excel.sheet_names[1]
score_df = data_excel.parse(sheet_name)
score_df

,グループID,全体,具体性,工夫,適切性
0,1,2,3,3,2
1,2,2,2,2,2
2,3,2,3,2,3
3,4,3,3,3,3
4,5,3,3,3,3
5,6,2,2,2,2
6,7,1,1,1,2
7,8,3,3,3,3
8,9,2,2,3,2
9,10,3,3,3,3


In [5]:
score = score_df.values
score

array([[ 1,  2,  3,  3,  2],
       [ 2,  2,  2,  2,  2],
       [ 3,  2,  3,  2,  3],
       [ 4,  3,  3,  3,  3],
       [ 5,  3,  3,  3,  3],
       [ 6,  2,  2,  2,  2],
       [ 7,  1,  1,  1,  2],
       [ 8,  3,  3,  3,  3],
       [ 9,  2,  2,  3,  2],
       [10,  3,  3,  3,  3],
       [11,  2,  2,  2,  2],
       [12,  1,  1,  2,  2],
       [13,  2,  2,  3,  2],
       [14,  3,  3,  3,  3],
       [15,  1,  1,  2,  2],
       [16,  1,  1,  1,  2],
       [17,  1,  1,  1,  1],
       [18,  2,  2,  2,  2],
       [19,  1,  1,  2,  1],
       [20,  3,  2,  3,  2],
       [21,  1,  1,  1,  2],
       [22,  2,  2,  3,  2],
       [23,  1,  1,  1,  2],
       [24,  1,  1,  1,  1],
       [25,  2,  2,  2,  3],
       [26,  2,  2,  2,  3],
       [27,  2,  3,  2,  2],
       [28,  2,  2,  3,  1],
       [29,  1,  1,  2,  2],
       [30,  1,  1,  1,  2],
       [31,  2,  2,  1,  3],
       [32,  2,  2,  2,  3],
       [33,  2,  2,  3,  2],
       [34,  1,  1,  1,  2],
       [35,  1

In [6]:
group = dict(Counter(data[:, 0]))
group_keys = np.array(list(group))
print("group(dict):", group)
print("group(key):", group_keys)

epi = dict(Counter(data[:, 2]))
epi_keys = np.array(list(epi))
print("epistemic(dict):", epi)
print("epistemic(key):", epi_keys)

arg = dict(Counter(data[:, 3]))
arg_keys = np.array(list(arg))
print("argumentation(dict):", arg)
print("argumentation(key):", arg_keys)

coo = dict(Counter(data[:, 4]))
coo_keys = np.array(list(coo))
print("coordination(dict):", coo)
print("coordination(key):", coo_keys)

soc = dict(Counter(data[:, 5]))
soc_keys = np.array(list(soc))
print("social(dict):", soc)
print("social(key):", soc_keys)

group(dict): {1: 54, 2: 50, 3: 47, 4: 86, 5: 62, 6: 107, 7: 61, 8: 66, 9: 98, 10: 57, 11: 29, 12: 58, 13: 61, 14: 28, 15: 61, 16: 53, 17: 59, 18: 79, 19: 74, 20: 49, 21: 44, 22: 59, 23: 76, 24: 21, 25: 41, 26: 70, 27: 67, 28: 73, 29: 117, 30: 38, 31: 36, 32: 64, 33: 50, 34: 35, 35: 77, 36: 60, 37: 69, 38: 30, 39: 83, 40: 90, 41: 47, 42: 34, 43: 66, 44: 51, 45: 60, 46: 46}
group(key): [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
epistemic(dict): {'off task': 1094, 'on task': 1649}
epistemic(key): ['off task' 'on task']
argumentation(dict): {'non-argumentative moves': 1649, 'simple claim': 1073, 'grounded claim': 21}
argumentation(key): ['non-argumentative moves' 'simple claim' 'grounded claim']
coordination(dict): {'blank': 2385, 'technical coordination': 348, 'proceedings': 10}
coordination(key): ['blank' 'technical coordination' 'proceedings']
social(dict): {'externalization': 2192, 'quick 

In [7]:
f=open('data/excel2/temp.csv','a')

for g in group_keys:
    g_data = data[(data[:, 0] == g)]
    g_name = np.array(list(dict(Counter(g_data[:,1]))))
    g_matrix = np.zeros(45).reshape(3,15)
    for n,i in zip(g_name,range(g_name.size)):
        e = np.sum(epi_keys == g_data[g_data[:, 1] == n][:,[2]], axis=0)
        g_matrix[[i],:2] = e
        g_matrix[[i],[2]]= sum(e)
        a = np.sum(arg_keys == g_data[g_data[:, 1] == n][:,[3]], axis=0)
        g_matrix[[i],3:6] = a
        g_matrix[[i],[6]]= sum(a)
        c = np.sum(coo_keys == g_data[g_data[:, 1] == n][:,[4]], axis=0)
        g_matrix[[i],7:10] = c
        g_matrix[[i],[10]]= sum(c)    
        s = np.sum(soc_keys == g_data[g_data[:, 1] == n][:,[5]], axis=0)
        g_matrix[[i],11:14] = s
        g_matrix[[i],[14]]= sum(s)     
    count_m = np.vstack([np.sum(g_matrix, axis=0),g_matrix])
    score_m = np.vstack([score[(score[:, 0] == g)][:,1:],np.zeros(12).reshape(3,4)])
    group_m = np.hstack([score_m,count_m])
    np.savetxt(f, group_m, delimiter=",", fmt='%i')
f.close()